<a href="https://colab.research.google.com/github/junweilam/ICT3104_T20/blob/main/ICT3104_Team20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing

**Installing of necessary dependencies**

In [ ]:
!pip install ipywidgets==7.6.3
!pip install pyyaml
!pip install opencv-python-headless

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

**Installation for Debugger**

In [ ]:
#Debugger
!pip install -Uqq ipdb
import ipdb

**Debugger Switch (On/Off)**

In [ ]:
# Turn On Debugger
%pdb on

# Turn Off Debugger
# %pdb off

# **Uploading Of Input Video File in .mp4 format** (T20-1)
1.   Run the cell and click the "Upload" button to upload a video in .mp4 format
2.   Once file is selected, click "Save MP4" button to save the .mp4 file
3.   File is saved when progress bar is 100% green and "Save MP4" button is changed to "MP4 Saved!"



In [ ]:
# Testing.ipynb

import ipywidgets as widgets
from IPython.display import display
import os
import shutil
import asyncio
import platform
import time
from base64 import b64encode


# Create a file uploader widget
file_upload = widgets.FileUpload(
    accept='.mp4',
    multiple=False
)

# Create a button widget
save_button = widgets.Button(
    description="Save MP4",
    disabled=False,
    button_style='success'
)

# Create a progress widget
progress = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    bar_style="success"
)

def update_progress_bar():
  progress.value += 25
  time.sleep(1)

def save_mp4(b):
    if file_upload.value:
        # Get the uploaded file
        uploaded_file = list(file_upload.value.values())[0]

        update_progress_bar()

        # Specify the destination folder to save the file
        destination_folder = './input/'

        update_progress_bar()

        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        update_progress_bar()

        with open(os.path.join(destination_folder, uploaded_file['metadata']['name']), 'wb') as f:
            f.write(uploaded_file['content'])

        update_progress_bar()

        # Disable the file uploaded and update the button text
        file_upload.disabled = True
        save_button.description = "MP4 Saved!"
    else:
        print("Please upload an MP4 file first")

display(file_upload, save_button, progress)

save_button.on_click(save_mp4)

# **Viewing of uploaded videos** (T20-21)

1.   Select a video uploaded from the dropdown list
2.   Click the "Play Video" button to display and view the video

In [ ]:
# Testing .ipynb
import os
from IPython.display import Video, display, HTML
import ipywidgets as widgets


# Path to the input directory
input_directory = './input'
video_files = []
if os.path.exists(input_directory):
  # Get a list of video files in the input directory
  video_files = [file for file in os.listdir(input_directory) if file.endswith('.mp4')]

# Create a dropdown widget with video file options
video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select Video:'
)


# Create a button widget for playing the selected video
play_button = widgets.Button(description="Play Video")

# Function to handle button click event
def play_video(event):
    selected_video = video_dropdown.value
    video_path = os.path.join(input_directory, selected_video)
    # display(Video(video_path, width=640, height=360))  # Adjust width and height as needed

    # For Google Collab
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""<video width=400 controls><source src="%s" type="video/mp4"></video>""" % data_url))


# Attach the function to the button's click event
play_button.on_click(play_video)

# Display the dropdown and button widgets
display(video_dropdown)
display(play_button)

# Create a pose skeleton(MMPose) of the character in the video doing the same action. (T20-4)

In [ ]:
# %pip install -U openmim
# !mim install mmengine
# !mim install "mmcv>=2.0.0"
# !mim install "mmdet>=3.0.0"

###Install MMPose from source

In [ ]:
# %cd /content
# !git clone https://github.com/open-mmlab/mmpose.git
# %cd mmpose
# !pip install -e .

### Run Demo (2D Human Whole-Body Pose Estimation with Inferencer)

❗ Rename the mp4 to your video accordingly for now

In [ ]:
# %cd /content/mmpose

# !python demo/inferencer_demo.py ../input/man-surfing.mp4 --black-background --pose2d human --vis-out-dir vis_results/posetrack18

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import os

# Define a function to run the pose estimation
def run_pose_estimation(video_path, black_background, pose2d, vis_out_dir):
    cmd = f"python demo/inferencer_demo.py {video_path} --black-background --pose2d {pose2d} --vis-out-dir {vis_out_dir}"
    if black_background:
        cmd += " --black-background"
    os.system(cmd)

# Specify the input directory where your video files are located
input_directory = '../input'

# Check if the input directory exists
if os.path.exists(input_directory):
    # Get a list of video files in the input directory
    video_files = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith('.mp4')]
else:
    video_files = []

# Create a dropdown widget to select a video
video_selector = widgets.Dropdown(
    options={os.path.basename(video_file): video_file for video_file in video_files},
    description='Select Video:',
)


# Create other widgets for additional options
black_background_checkbox = widgets.Checkbox(value=True, description='Black Background')
pose2d_selector = widgets.Dropdown(
    options=['human', 'other_pose_option'],
    description='Pose2D:',
)
vis_out_dir_text = widgets.Text(value='vis_results/posetrack18', description='Output Directory:')

# Create a button to trigger the pose estimation
run_button = widgets.Button(description='Run Pose Estimation')

# Define an event handler for the button click
def run_button_click(b):
    video_path = video_selector.value
    black_background = black_background_checkbox.value
    pose2d = pose2d_selector.value
    vis_out_dir = vis_out_dir_text.value
    run_pose_estimation(video_path, black_background, pose2d, vis_out_dir)

run_button.on_click(run_button_click)

# Display the widgets
display(video_selector, black_background_checkbox, pose2d_selector, vis_out_dir_text, run_button)

In [ ]:
# @title Generating Skeleton based on video selected
# Pose - Pose Skeleton Creation
# %pip install ipywidgets
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from base64 import b64encode
import time  # Import the time module
import subprocess

%cd /content
if not os.path.exists('./mmpose'):
  %pip install -U openmim
  !mim install mmengine
  !mim install "mmcv>=2.0.0"
  !mim install "mmdet>=3.0.0"
  !git clone https://github.com/open-mmlab/mmpose.git
  %cd mmpose
  !pip install -e .
  %cd /content

# add dropdown to allow selection of poses
list_of_poses = []
input_folder = "./input"
if os.path.exists(input_folder):
  for file in os.listdir(input_folder):
    if file.endswith(".mp4"):
        list_of_poses.append(file)

# Widget Dropdown
pose_dropdown = widgets.Dropdown(
    options=list_of_poses,
    description='Select a MP4 File:'
)

section_label = widgets.HTML(value='<h2 style="font-size: 20px;">Generating Skeleton </h2>')

# Create a button widget
button = widgets.Button(description="Generate Skeleton")

def display_video(video_path):
    with open(video_path, "rb") as f:
        video_data = f.read()
        video_base64 = b64encode(video_data).decode("utf-8")
        video_html = f'<video controls width="400"><source src="data:video/mp4;base64,{video_base64}" type="video/mp4"></video>'
        display(HTML(video_html))
    display_message('Video Generated!')

def display_loading():
    loading_message = HTML('<p>Video Generating... Please Wait...</p>')
    display(loading_message)
    # Simulate video generation with a sleep
    time.sleep(3)  # Sleep for 3 seconds [Will help to let users know the video is generating.]

# Annouce to user that video is done and can input another prompt
def display_message(message):
    loading_message = HTML('<p>' + message + '</p>')
    display(loading_message)

def save_skeleton(): # TODO! FIX COMMAND
    # Specify the full path to the input video and output directory
    input_video = '../input/' + pose_dropdown.value
    output_dir = 'vis_results/posetrack18'

    !python demo/inferencer_demo.py {input_video} --black-background --pose2d human --vis-out-dir {output_dir}
    print(f"Skeleton created and saved to: /content/mmpose/{output_dir + '/' +  pose_dropdown.value}")

# Define a function to handle the button click event
def get_input(event):
    print(f"User's prompt: {input_text.value}")
    display_loading()
    display_video("input/" + pose_dropdown.value)
    display_message('This is the video you have selected!')

    %cd /content/mmpose
    save_skeleton()
    display_message('Please View Skeleton at T20-11.')

# Attach the function to the button's click event
button.on_click(get_input)

# Display the widgets
display(section_label, pose_dropdown, button)

# **Creating of YAML files for the input video** (T20-12)

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display
import yaml

input_directory = '/content/mmpose/vis_results/posetrack18'
video_files = []
if os.path.exists(input_directory):
  # Get a list of video files in the input directory
  video_files = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith('.mp4')]

# Create input widgets for configuration parameters
config_name_input = widgets.Text(
    value="configs/man-skiing.yaml",
    description="Config Name:",
)

train_video_path_input = widgets.Dropdown(
    options=video_files,
    description='Select Video:'
)

train_prompt_input = widgets.Text(
    value="a man is skiing",
    description="Train Prompt:",
)

video_length_input = widgets.IntSlider(
    value=8,
    min=1,
    max=30,
    step=1,
    description="Video Length:",
)

width_input = widgets.IntSlider(
    value=512,
    min=128,
    max=1024,
    step=64,
    description="Width:",
)

height_input = widgets.IntSlider(
    value=512,
    min=128,
    max=1024,
    step=64,
    description="Height:",
)

learning_rate_input = widgets.FloatLogSlider(
    value=3e-5,
    base=10,
    min=-5,
    max=0,
    step=0.1,
    description="Learning Rate:",
)

train_steps_input = widgets.IntSlider(
    value=300,
    min=100,
    max=1000,
    step=100,
    description="Train Steps:",
)

# Create a progress widget
progress = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    bar_style="success"
)

def update_progress_bar():
  progress.value += 35
  time.sleep(1)


# Create a button to generate the configuration
generate_config_button = widgets.Button(
    description="Generate Configuration",
)

# Function to generate the configuration
def generate_config(event):
    # Get the full path of the configuration file
    config_file_path = config_name_input.value

    update_progress_bar()

    # Ensure that the directory for the configuration file exists
    config_directory = os.path.dirname(config_file_path)
    os.makedirs(config_directory, exist_ok=True)



    config = {
        "train_data": {
            "video_path": train_video_path_input.value,
            "prompt": train_prompt_input.value,
            "n_sample_frames": video_length_input.value,
            "width": width_input.value,
            "height": height_input.value,
            "sample_start_idx": 0,
            "sample_frame_rate": 2,
        },
        "validation_data": {
            "prompts": [
                "mickey mouse is skiing on the snow",
                "spider man is skiing on the beach, cartoon style",
                "wonder woman, wearing a cowboy hat, is skiing",
                "a man, wearing pink clothes, is skiing at sunset",
            ],
            "video_length": video_length_input.value,
            "width": width_input.value,
            "height": height_input.value,
            "num_inference_steps": 20,
            "guidance_scale": 12.5,
            "use_inv_latent": True,
            "num_inv_steps": 50,
        },
        "learning_rate": learning_rate_input.value,
        "train_batch_size": 1,
        "max_train_steps": train_steps_input.value,
        "checkpointing_steps": 1000,
        "validation_steps": 100,
        "trainable_modules": [
            "attn1.to_q",
            "attn2.to_q",
            "attn_temp",
        ],
        "seed": 33,
        "mixed_precision": "fp16",
        "use_8bit_adam": False,
        "gradient_checkpointing": True,
        "enable_xformers_memory_efficient_attention": True,
    }
    generate_config_button.description = "Completed"
    generate_config_button.disabled = True

    update_progress_bar()

    # Save the configuration as a YAML file
    with open(config_file_path, 'w') as config_file:
        yaml.dump(config, config_file)

    update_progress_bar()
# Attach event handler to the generate button
generate_config_button.on_click(generate_config)

# Display input widgets and generate button
display(config_name_input)
display(train_video_path_input)
display(train_prompt_input)
display(video_length_input)
display(width_input)
display(height_input)
display(learning_rate_input)
display(train_steps_input)
display(generate_config_button)
display(progress)

# **Training Model Section** (T20-12)

In [ ]:
import os
import ipywidgets as widgets
import yaml
from IPython.display import display
from tqdm import tqdm  # For the loading bar

# Specify your dataset directory in Google Colab
dataset_dir = '/content/dataset'

# Get a list of subfolders in the dataset directory
subfolders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]

# Create a dropdown widget to choose a subfolder
subfolder_dropdown = widgets.Dropdown(
    options=subfolders,
    description='Select Subfolder:'
)

# Display the dropdown to the user
display(subfolder_dropdown)

# Input field for specifying the model name
model_name_input = widgets.Text(
    placeholder='Enter Model Name',
    description='Model Name:'
)

# Display the input field for the model name
display(model_name_input)

# Initialize an empty list to store video properties
video_properties_list = []

# Define a function to generate a YAML configuration based on video properties
def generate_yaml_config(video_properties_list, model_name, subfolder):
    if not video_properties_list:
        print("No video properties found. Training aborted.")
        return

    config = {
        'pretrained_model_path': "./checkpoints/stable-diffusion-v1-4",
        'output_dir': "output",
        'train_data': {
            'video_path': "no path",
            'prompt': "None",
            'n_sample_frames': 12,
            'width': 512,
            'height': 512,
            'sample_start_idx': 0,
            'sample_frame_rate': 4,
            'dataset_set': "train"
        },
        'validation_data': {
            'prompts': [
                'A Iron man on the beach',
                'A Spider man on the snow',
                'A Superman on the street',
                'A boy in the forest'
            ],
            'video_length': 24,
            'width': 512,
            'height': 512,
            'num_inference_steps': 50,
            'guidance_scale': 12.5,
            'use_inv_latent': False,
            'num_inv_steps': 50,
            'dataset_set': "val"
        },
        'learning_rate': 3e-5,
        'train_batch_size': 1,
        'max_train_steps': 5000,
        'checkpointing_steps': 1000,
        'validation_steps': 100,
        'trainable_modules': [
            "attn1.to_q",
            "attn2.to_q",
            "attn_temp",
            "conv_temporal"
        ],
        'skeleton_path': './pose_example/vis_kun_pose2.mov',
        'seed': 33,
        'mixed_precision': 'no',
        'use_8bit_adam': False,
        'gradient_checkpointing': False,
        'enable_xformers_memory_efficient_attention': True
    }

    # Define the output YAML file path
    output_yaml_path = os.path.join('/content/configs', f'{model_name}_{subfolder}_config.yaml')

    # Save the configuration dictionary to a YAML file
    with open(output_yaml_path, 'w') as yaml_file:
        yaml.dump(config, yaml_file)

    print(f'Configuration saved to {output_yaml_path}')

# Define a button to trigger training
train_button = widgets.Button(
    description='Train Model'
)

# Function to handle button click event
def train_button_click(b):
    subfolder = subfolder_dropdown.value
    model_name = model_name_input.value

    # Iterate through videos in the selected subfolder
    subfolder_path = os.path.join(dataset_dir, subfolder)
    video_files = [f for f in os.listdir(subfolder_path) if f.endswith('.mp4')]

    for video_file in video_files:
        video_path = os.path.join(subfolder_path, video_file)

        # Extract video properties and add to the list
        video_properties = {
            'video_path': video_path,
            'width': 512,  # Replace with actual video width
            'height': 512,  # Replace with actual video height
            'sample_frame_rate': 30,  # Replace with actual frame rate
            'duration': 300,  # Replace with actual video duration
            'n_sample_frames': 12,
            'sample_start_idx': 0
        }
        video_properties_list.append(video_properties)

    # Generate YAML config based on aggregated video properties
    generate_yaml_config(video_properties_list, model_name, subfolder)
    # Add your training logic here using the generated YAML config for all videos

    # Simulate training with a loading bar (replace this with your actual training code)
    num_epochs = config.get("num_epochs", 10)  # Number of training epochs
    for epoch in range(num_epochs):
        # Create a loading bar
        progress_bar = tqdm(total=num_epochs, position=0, leave=True)
        progress_bar.set_description(f'Epoch {epoch + 1}/{num_epochs}')
        for _ in range(10):  # Simulate training steps
            progress_bar.update(1)  # Update the loading bar
            import time
            time.sleep(0.1)  # Simulate a training step
        progress_bar.close()
        # Add "Training completed" message
    print('Training completed')

# Attach the click event handler to the training button
train_button.on_click(train_button_click)

# Display the training button
display(train_button)

TESTING FOR YAML

In [ ]:
import yaml

# Load the YAML configuration file
config_file_path = "/content/drive/My Drive/man-skiing.yaml"  # Replace with the actual path
with open(config_file_path, 'r') as config_file:
    config = yaml.safe_load(config_file)

In [ ]:
# Example: Use OpenCV to read and process the video
from google.colab.patches import cv2_imshow
import cv2  # Import the OpenCV library

video_path = config["train_data"]["video_path"]

cap = cv2.VideoCapture(video_path)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame here (e.g., apply image processing)

    # Display or save the processed frame
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Specify Text Prompt as input to GenAI model and see inference results in the form of output videos with captions depicting action/activity, so that I can access model's quality. (T20-11)

In [ ]:
# @title Viewing Video Reference based on config and prompt
# Pose - Viewing Skeleton Reference
# %pip install ipywidgets
%cd /content/mmpose
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from base64 import b64encode
import time  # Import the time module
import subprocess

# add dropdown to allow selection of poses
list_of_poses = []
input_folder = "../input"
if os.path.exists(input_folder):
  for file in os.listdir(input_folder):
    if file.endswith(".mp4"):
        list_of_poses.append(file)

# Widget Dropdown
pose_dropdown = widgets.Dropdown(
    options=list_of_poses,
    description='Select a MP4 File:'
)

section_label = widgets.HTML(value='<h2 style="font-size: 20px;">View Reference as Video</h2>')

input_label = widgets.Label(value="Enter your prompt:") # Label for the Input Field Widget
input_text = widgets.Text(placeholder='eg. A Man is skiing') # Input Field Widget for User to enter prompt
prompt_input_box = widgets.HBox([input_label, input_text])

# Create a button widget
button = widgets.Button(description="View Reference")

def display_video(video_path):
    with open(video_path, "rb") as f:
        video_data = f.read()
        video_base64 = b64encode(video_data).decode("utf-8")
        video_html = f'<video controls width="400"><source src="data:video/mp4;base64,{video_base64}" type="video/mp4"></video>'
        display(HTML(video_html))
    display_success('Video Generated!')

def display_loading():
    loading_message = HTML('<p>Video Generating... Please Wait...</p>')
    display(loading_message)
    # Simulate video generation with a sleep (replace this with your actual video generation logic)
    time.sleep(3)  # Sleep for 3 seconds [Will help to let users know the video is generating.]

# Annouce to user that video is done and can input another prompt
def display_success(message):
    loading_message = HTML('<p>' + message + '</p>')
    display(loading_message)

# Define a function to handle the button click event
def get_input(event):
    print(f"User's prompt: {input_text.value}")
    display_loading()
    display_video("input/" + pose_dropdown.value)
    %cd /content/mmpose
    display_video('vis_results/posetrack18/' + pose_dropdown.value)

# Attach the function to the button's click event
button.on_click(get_input)

# Display the widgets
display(section_label, pose_dropdown, prompt_input_box, button)